In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['train.csv', 'test.csv', 'sample_submission.csv']


In [2]:
data = pd.read_csv("../input/train.csv")
data_file = pd.read_csv("../input/test.csv")
data.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,pixel25,pixel26,pixel27,pixel28,pixel29,pixel30,pixel31,pixel32,pixel33,pixel34,pixel35,pixel36,pixel37,pixel38,...,pixel744,pixel745,pixel746,pixel747,pixel748,pixel749,pixel750,pixel751,pixel752,pixel753,pixel754,pixel755,pixel756,pixel757,pixel758,pixel759,pixel760,pixel761,pixel762,pixel763,pixel764,pixel765,pixel766,pixel767,pixel768,pixel769,pixel770,pixel771,pixel772,pixel773,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [3]:
labels = data['label']
col = list(data.columns)
col.remove('label')


In [4]:
from keras.utils.np_utils import to_categorical
data = data[col].values.reshape(-1,28,28,1)
labels = to_categorical(labels,num_classes=10)
data_file = data_file.values.reshape(-1,28,28,1)
labels


Using TensorFlow backend.


array([[0., 1., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.]], dtype=float32)

In [5]:
print(data.shape)
print(labels.shape)
print(data_file.shape)

(42000, 28, 28, 1)
(42000, 10)
(28000, 28, 28, 1)


In [6]:
from sklearn.model_selection import train_test_split
data_train,data_test,label_train,label_test = train_test_split(data,labels,train_size=0.9)

from keras.preprocessing.image import ImageDataGenerator

/opt/conda/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [7]:
from keras.models import Sequential 
from keras.layers import Dense,Conv2D,Flatten,MaxPooling2D,Dropout,GlobalAveragePooling2D

model = Sequential()
model.add(Conv2D(filters=16, kernel_size=3, padding='same', activation='relu', input_shape=(28, 28, 1)))
model.add(Conv2D(filters=16, kernel_size=3, padding='same', activation='relu'))
model.add(Dropout(0.1))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(Conv2D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))
model.add(Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'))
model.add(Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'))
model.add(Dropout(0.3))
model.add(GlobalAveragePooling2D())
model.add(Dense(500, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(10, activation='softmax'))
model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [8]:
data_generator = ImageDataGenerator(rotation_range=20,zoom_range=0.15, 
                                    width_shift_range=0.2,height_shift_range=0.2, shear_range=0.15,fill_mode="nearest")
data_generator.fit(data_train)
data_generated = data_generator.flow(data,labels,batch_size=32)

In [9]:
history = model.fit_generator(data_generated,epochs=10,steps_per_epoch=len(data)//32)
pred = model.predict_classes(data_file)

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
1312/1312 [==============================] - 18s 14ms/step - loss: 0.5247 - acc: 0.8278
Epoch 2/10
1312/1312 [==============================] - 16s 12ms/step - loss: 0.1339 - acc: 0.9611
Epoch 3/10
1312/1312 [==============================] - 16s 12ms/step - loss: 0.1108 - acc: 0.9698
Epoch 4/10
1312/1312 [==============================] - 16s 12ms/step - loss: 0.1005 - acc: 0.9717
Epoch 5/10
1312/1312 [==============================] - 16s 12ms/step - loss: 0.1005 - acc: 0.9733
Epoch 6/10
1312/1312 [==============================] - 16s 12ms/step - loss: 0.1045 - acc: 0.9721
Epoch 7/10
1312/1312 [==============================] - 16s 12ms/step - loss: 0.1039 - acc: 0.9734
Epoch 8/10
1312/1312 [==============================] - 16s 12ms/step - loss: 0.1121 - acc: 0.9716
Epoch 9/10
1312/1312 [==============================] - 16s 12ms/step - loss: 0.1129 - acc: 0.9712
Epoch 10/10
1312/1312 [==============================] - 16s 

In [10]:
pd.DataFrame({"ImageId":list(range(1,len(data_file)+1)),"Label":pred}).to_csv("output_NN.csv",index=False)